In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML
import warnings

data = pd.read_csv('../data/ERP_data.csv')
labels = list(data.columns.values)
del labels[0]
del labels[0]

targets = data['Phenotype']
del data['Subject']
del data['Phenotype']

In [2]:
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn import cross_validation

folds = 10

imp = preprocessing.Imputer()
data = imp.fit_transform(data, targets)
data = preprocessing.scale(data)
anova_selection = feature_selection.SelectKBest(feature_selection.f_classif, k=11)

In [3]:
from sklearn import ensemble
from sklearn import pipeline
from sklearn import metrics

adaboost_model = ensemble.AdaBoostClassifier()

clf = pipeline.make_pipeline(anova_selection, adaboost_model)

accuracies = []
precisions = []
recalls = []
fscores = []
confusions = []
warnings.filterwarnings("ignore")
for i in range(200):
    cross_val = cross_validation.StratifiedKFold(targets, n_folds=folds, shuffle=True)
    preds = cross_validation.cross_val_predict(clf, data, targets, cv=cross_val)
    accuracy = metrics.accuracy_score(targets, preds)
    precision, recall, fscore, support = metrics.precision_recall_fscore_support(
        targets, preds, average='binary', pos_label='AD')
    confusion = metrics.confusion_matrix(targets, preds, labels=['HC', 'AD'])
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    fscores.append(fscore)
    confusions.append(confusion)

print("Accuracy: {0}, with std: {1}".format(np.mean(accuracies), np.std(accuracies)))
print("Precision: {0}".format(np.mean(precisions)))
print("Recall: {0}".format(np.mean(recalls)))
print("F1 Score: {0}".format(np.mean(fscores)))
print("Confusion Matrix:\n   HC     AD\n{0}".format(np.mean(confusions, axis=0)))

Accuracy: 0.6500251256281407, with std: 0.026101150236833733
Precision: 0.6399460973602927
Recall: 0.6797979797979798
F1 Score: 0.6589426921630583
Confusion Matrix:
   HC     AD
[[ 62.055  37.945]
 [ 31.7    67.3  ]]
